In [1]:
import pandas as pd
filename = 'C:/Users/diaga/OneDrive/Documents/Springboard/Online Retail.xlsx'
data = pd.read_excel(filename)
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [3]:
data.dtypes

InvoiceNo              object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
UnitPrice             float64
CustomerID            float64
Country                object
dtype: object

In [46]:
#len(data[data.UnitPrice < 0 ])
len(data[data.CustomerID.isna()])
#data[data.CustomerID.isna()].groupby('Country').count()
#data[data.CustomerID.isna()].groupby(['Country','Description'])['InvoiceNo'].count().sort_values(ascending=False).head(60)

135080

In [33]:
len(data[data.UnitPrice < 0 ])
print(data[data.UnitPrice < 0 ])

       InvoiceNo StockCode      Description  Quantity         InvoiceDate  \
299983   A563186         B  Adjust bad debt         1 2011-08-12 14:51:00   
299984   A563187         B  Adjust bad debt         1 2011-08-12 14:52:00   

        UnitPrice  CustomerID         Country  
299983  -11062.06         NaN  United Kingdom  
299984  -11062.06         NaN  United Kingdom  


In [21]:
len(data)

541909

In [47]:
data_clean = data[data['CustomerID'].notna()]
len(data_clean)

406829

In [51]:
data_clean[['StockCode','Quantity','Country','UnitPrice']].nunique()

StockCode    3684
Quantity      436
Country        37
UnitPrice     620
dtype: int64

In [92]:
data_clean.groupby(['StockCode'])['UnitPrice'].nunique().sort_values(ascending=False).head()

StockCode
M       184
D        75
POST     71
DOT      16
CRUK     16
Name: UnitPrice, dtype: int64

In [78]:
test = data_clean.groupby(['StockCode'])['UnitPrice'].nunique()
test[test>1].count()

2642

In [ ]:
#import matplotlib.pyplot as plt
#_ = plt.hist(data_clean['UnitPrice'])
#plt.show()

In [86]:
data_mean = data_clean.groupby('CustomerID').mean()
data_mean.columns

Index(['Quantity', 'UnitPrice'], dtype='object')

In [89]:
data_count = data_clean.groupby('CustomerID').nunique()
data_count.info()

<class 'pandas.core.frame.DataFrame'>
Float64Index: 4372 entries, 12346.0 to 18287.0
Data columns (total 8 columns):
InvoiceNo      4372 non-null int64
StockCode      4372 non-null int64
Description    4372 non-null int64
Quantity       4372 non-null int64
InvoiceDate    4372 non-null int64
UnitPrice      4372 non-null int64
CustomerID     4372 non-null int64
Country        4372 non-null int64
dtypes: int64(8)
memory usage: 307.4 KB


In [93]:
data_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 406829 entries, 0 to 541908
Data columns (total 8 columns):
InvoiceNo      406829 non-null object
StockCode      406829 non-null object
Description    406829 non-null object
Quantity       406829 non-null int64
InvoiceDate    406829 non-null datetime64[ns]
UnitPrice      406829 non-null float64
CustomerID     406829 non-null float64
Country        406829 non-null object
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 47.9+ MB


In [98]:
from datetime import datetime
data_clean['Invoice_DAY'] = pd.to_datetime(data_clean['InvoiceDate']).dt.strftime('%m/%d/%Y')
data_clean.sort_values(ashead()

C:\Users\diaga\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Invoice_DAY
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,12/01/2010
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,12/01/2010
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,12/01/2010
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,12/01/2010
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,12/01/2010
